<a href="https://colab.research.google.com/github/SinaRampe/applications-with-LangChain/blob/main/Chroma_DB_Multi_pdf_retriever_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip -q install langchain openai tiktoken chromadb pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.8/248.8 kB 8.2 MB/s eta 0:00:00


# LangChain multi-doc retriever with ChromaDB

***New Points***
- Multiple Files
- ChromaDB
- Source info 
- gpt-3.5-turbo API

## Setting up LangChain 


In [16]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [11]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader

## Load multiple and process documents

In [9]:
loader = PyPDFDirectoryLoader("data/")
raw_documents = loader.load()
print(f"loaded {len(raw_documents) } documents")

loaded 466 documents


In [12]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(raw_documents)

In [13]:
len(texts)

1095

In [14]:
texts[3]

Document(page_content='Complexity made simple. This is a rare and pr ecious book  about  NLP,\ntransformers, and t he growing ecosystem around t hem, Hugging Face.\nWhether these are still buzzwords to you or  you al ready have a solid\ngrasp of it all, the authors will navigate you w ith hum or, scientific rigor,\nand pl enty of code examples into the deepest secrets of the coolest\ntechnology around. F rom “off-the-shelf pretrained” to “from-scratch\ncustom” models, and f rom performance to missing labels issues, the\nauthors addr ess practically every real-life struggle of a ML engineer\nand pr ovide state-of-the-art solutions, making this book  destined to\ndictate the standar ds in the field for years to come.\n—Luca Perrozzi, PhD, Data Science and Machine Learning\nAssociate Manager at Accenture', metadata={'source': 'data/Natural Language Processing with Transformers Building Language Applications with Hugging Face.pdf', 'page': 2})

## create the DB

In [18]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## here we are using OpenAI embeddings but in future we will swap out to local embeddings
embedding = OpenAIEmbeddings(disallowed_special=())

vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [19]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [20]:
# Now we can load the persisted database from disk, and use it as normal. 
vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding)

## Make a retriever

In [21]:
retriever = vectordb.as_retriever()

In [22]:
docs = retriever.get_relevant_documents("What is NLP?")

In [23]:
len(docs)

4

In [24]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [25]:
retriever.search_type

'similarity'

In [26]:
retriever.search_kwargs

{'k': 2}

## Make a chain

In [27]:
# create the chain to answer questions 
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(), 
                                  chain_type="stuff", 
                                  retriever=retriever, 
                                  return_source_documents=True)

In [28]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [29]:
# full example
query = "What is NLP?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 NLP stands for natural language processing, which is the use of algorithms to process and understand natural language. It encompasses the whole realm of tasks such as text classification, summarization, translation, question answering, chatbots, natural language understanding, and more.


Sources:
data/Natural Language Processing with Transformers Building Language Applications with Hugging Face.pdf
data/Natural Language Processing with Transformers Building Language Applications with Hugging Face.pdf


In [30]:
# break it down
query = "What is NLP?"
llm_response = qa_chain(query)
# process_llm_response(llm_response)
llm_response

{'query': 'What is NLP?',
 'result': ' NLP stands for natural language processing, and is the application of technology to the interpretation and manipulation of natural language. It encompasses the whole realm of tasks from text classification to summarization, translation, question answering, chatbots, natural language understanding (NLU), and more.',
 'source_documents': [Document(page_content='Copilot system is helping me write these lines: you’ll never know how much\nI really wrote.\nThe revolution goes far beyond text generation. It encompasses the whole\nrealm of natural language processing (NLP), from text classification to\nsummarization, translation, question answering, chatbots, natural language\nunderstanding (NLU), and more. Wherever there’s language, speech or text,\nthere’s an application for NLP. You can already ask your phone for\ntomorrow’s weather, or chat with a virtual help desk assistant to troubleshoot\na problem, or get meaningful results from search engines tha

In [31]:
query = "What is Huggingface"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Huggingface is an ecosystem of tools and resources that helps developers quickly and easily create models for natural language processing (NLP) tasks. It hosts over 20,000 freely available models and provides filters for tasks, frameworks, datasets, and more.


Sources:
data/Natural Language Processing with Transformers Building Language Applications with Hugging Face.pdf
data/Natural Language Processing with Transformers Building Language Applications with Hugging Face.pdf


In [32]:
query = "What is generative ai?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Generative AI refers to AI technologies that are able to create content, such as text, images, and audio. It typically involves using deep learning models, such as transformers, to generate new content based on a given prompt.


Sources:
data/Natural Language Processing with Transformers Building Language Applications with Hugging Face.pdf
data/Natural Language Processing with Transformers Building Language Applications with Hugging Face.pdf


In [33]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

('similarity', <langchain.vectorstores.chroma.Chroma at 0x7f799c2b3640>)

In [34]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


## Deleteing the DB

In [35]:
!zip -r db.zip ./db

  adding: db/ (stored 0%)
  adding: db/chroma-embeddings.parquet (deflated 29%)
  adding: db/index/ (stored 0%)
  adding: db/index/id_to_uuid_66206206-1387-4ea7-a77f-7126acab2376.pkl (deflated 36%)
  adding: db/index/index_metadata_66206206-1387-4ea7-a77f-7126acab2376.pkl (deflated 5%)
  adding: db/index/uuid_to_id_66206206-1387-4ea7-a77f-7126acab2376.pkl (deflated 39%)
  adding: db/index/index_66206206-1387-4ea7-a77f-7126acab2376.bin (deflated 17%)
  adding: db/chroma-collections.parquet (deflated 50%)


In [36]:
# To cleanup, you can delete the collection
vectordb.delete_collection()
vectordb.persist()

# delete the directory
!rm -rf db/

## Starting again loading the db

restart the runtime

In [37]:
!unzip db.zip

Archive:  db.zip
   creating: db/
  inflating: db/chroma-embeddings.parquet  
   creating: db/index/
  inflating: db/index/id_to_uuid_66206206-1387-4ea7-a77f-7126acab2376.pkl  
  inflating: db/index/index_metadata_66206206-1387-4ea7-a77f-7126acab2376.pkl  
  inflating: db/index/uuid_to_id_66206206-1387-4ea7-a77f-7126acab2376.pkl  
  inflating: db/index/index_66206206-1387-4ea7-a77f-7126acab2376.bin  
  inflating: db/chroma-collections.parquet  


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [38]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [43]:
persist_directory = 'db'
embedding = OpenAIEmbeddings(disallowed_special=())

vectordb2 = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding,
                   )

retriever = vectordb2.as_retriever(search_kwargs={"k": 2})

In [41]:
# Set up the turbo LLM
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

In [42]:
# create the chain to answer questions 
qa_chain = RetrievalQA.from_chain_type(llm=turbo_llm, 
                                  chain_type="stuff", 
                                  retriever=retriever, 
                                  return_source_documents=True)

In [44]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [45]:
# full example
query = "How much money did Pando raise?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

There is no information provided about Pando raising money, so I don't know the answer to that question.


Sources:
data/Natural Language Processing with Transformers Building Language Applications with Hugging Face.pdf
data/Natural Language Processing with Transformers Building Language Applications with Hugging Face.pdf


### Chat prompts

In [46]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[0].prompt.template)

Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{context}


In [47]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[1].prompt.template)

{question}
